# Visualize the data

In this notebook, will construct a graph to visualize the data.

## Import the data

Importing the data using f# because I dunno how to access the filesystem with javascript.

In [12]:
type AlkgieV1SourceEntry = {
    Id: Guid
    SourceName: string
    EntityNameAlias: Option<string>
    Description: Option<string>
    Url: Option<string>
    Relations: Guid[]
}

type AlkgieV1Entity =
    {
        Id: Guid
        DisplayName: string
        SourceEntries: AlkgieV1SourceEntry[]
        EntityType: string // Changing this from a discriminated union to a string because can't serailize discriminated unions to Javascript
    }

#r "nuget:FSharp.Data"

open FSharp.Data

type DedupedData = JsonProvider<"../data/deduped/deduped-dataset.json">

let dedupedData : AlkgieV1Entity[] =
    DedupedData.Load("../data/deduped/deduped-dataset.json")
    |> Seq.map (fun x -> 
        { 
            Id = x.Id
            DisplayName = x.DisplayName
            SourceEntries = 
                x.SourceEntries |> Seq.map (fun se -> 
                    {
                        SourceName = se.SourceName
                        Id = se.Id
                        EntityNameAlias = se.EntityNameAlias
                        Description = se.Description
                        Url = se.Url
                        Relations = se.Relations |> Seq.toArray
                    }) |> Seq.toArray
            EntityType = x.EntityType 
        }) |> Seq.toArray


Installed Packages FSharp.Data, 6.4.0

### Define some javascript resources

Imports and helper functions to make coding in javascript easier inside of this notebook.

The circular replacer makes it so that javascript objects can be serialized even when they are recursive, which helps with debugging.
Using cheerio and marked, both javascript packages, parse the content into a queryable tree.

In [16]:
echarts = await import('https://cdnjs.cloudflare.com/ajax/libs/echarts/5.4.3/echarts.esm.min.js');

getCircularReplacer = function () {
    const ancestors = [];
    return function (key, value) {
      if (typeof value !== "object" || value === null) {
        return value;
      }
      // `this` is the object that value is contained in,
      // i.e., its direct parent.
      while (ancestors.length > 0 && ancestors.at(-1) !== this) {
        ancestors.pop();
      }
      if (ancestors.includes(value)) {
        return "[Circular]";
      }
      ancestors.push(value);
      return value;
    };
  }

## Render the graph

First we will render the html which we will use to register our charts, then in a separate cell we will initialize the graph using echarts.

In [17]:
<code id="fakeConsole"></code>
<div style="margin: 20px; display: flex;">
    <div id="main" style="width: 1200px;height:600px; background: purple;"></div>
</div>

Add interactivity to the chart. Note sometimes this code fails as it runs before the html dom has time to load. If the chart above is still just a purple square, try re-running the below cell. :)

In [18]:
#!set --value @fsharp:dedupedData --name dedupedData

// Create the ECharts instance
const myChart = echarts.init(document.getElementById('main'), 'dark');

// Define transformations
function getNode(data, i) {
    return {
        id: data.Id,
        name: data.DisplayName,
        x: i,
        y: i,
        label: {
            show: true//data.EntityType === 'Theme'
        },
        itemStyle: {
            color: data.EntityType === 'Theme' ? 'red' : 'blue'
        },
        draggable: true
    }
}

function getEdges(data) {
    const sourceEntries = data.SourceEntries;
    const relations = sourceEntries.map(entry => entry.Relations);
    return relations.flat().map(relation => {
        return {
            source: data.Id,
            target: relation
        }
    });
}

// Define the data for nodes and edges
const nodes = dedupedData.map(getNode); // remove theme filter to show all nodes
const edges = dedupedData.flatMap(getEdges);


// Create the option for the force-directed graph
const option = {
  tooltip: {},
  series: [
    {
      type: 'graph',
      layout: 'force',
      data: nodes,
      edges: edges,
      roam: true,
      force: {
        repulsion: 100,
        edgeLength: 100
      },
      label: {
        position: 'right',
        formatter: '{b}'
      },
    }
  ]
};

// Set the option and render the chart
myChart.setOption(option);
